# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 




## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the SQL questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


REVIEW
 total
 reviews
 reviews[x].rating
 reviews[x].time_created	
 
## SQL Questions:

- Which are the 5 most reviewed businesses? - REVIEWS
- What is the highest rating recieved in your data set and how many businesses have that rating? - RATINGS
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? P
PRICE $$$
- Return the text of the reviews for the most reviewed restaurant.  
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


In [3]:
#COMMIT WHEN INSERTING DATA

## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [565]:
import mysql.connector 

In [566]:
import config
db_name = 'Plumbs'


In [567]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = db_name

)
print(cnx)

In [568]:
cursor = cnx.cursor()

In [569]:
from mysql.connector import errorcode

In [292]:
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [293]:
# cursor.execute('DROP TABLE Plumb_biz;')

In [294]:
db_name = 'Plumbs'

TABLES = {}
TABLES["Plumb_biz"] = (
   "CREATE TABLE Plumb_biz ("
   "  biz_id varchar(100) NOT NULL PRIMARY KEY,"
   "  biz_name varchar(100) NOT NULL,"
   "  review_count int(16) NOT NULL,"
   "  rating int(7) NOT NULL"
   ") ENGINE=InnoDB")

TABLES['Plumb_Reviews'] = (
    "CREATE TABLE Plumb_Reviews ("
    "  rev_id int(71) NOT NULL,"
    "  rev_rating int(8) NOT NULL,"
    "  name varchar(40) NOT NULL,"
    "  user_id varchar(40) NOT NULL,"
    "  review_text varchar(5000) NOT NULL,"
    "  rev_date date NOT NULL,"
    "  biz_id int(40) NOT NULL,"
    "  PRIMARY KEY (rev_id)"
    ") ENGINE=InnoDB")

In [295]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

# cursor.close()
# cnx.close()

Creating table Plumb_biz: already exists.
Creating table Plumb_Reviews: already exists.


In [520]:
url = 'https://api.yelp.com/v3/businesses/search'
client_id = "kSJKHzaATCEw4bxY9tJGwA"
api_key = "xZce1e9e3JthFVYS0eQm1Aa6thcOEaQ2N2hzi3PJsJrv6ga4hinQMYsPWTBYQYRikD7sN9H7HQJV9sDPBoIf-m2Ko-0rw5ouSZhGgUtQDYA5_5hEipYddHf7Ct2xXXYx"

In [521]:
import json
import requests
import time

In [546]:
term = 'Plumbers'
location = 'Bushwick NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50,
                'offset' : 50
             }

In [308]:
# url = https://api.yelp.com/v3/businesses/{id}/reviews
# headers = {'Authorization': 'Bearer {}'.format(api_key)}
# response = requests.get(url, headers=headers, params=url_params)
   

In [547]:
def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

In [548]:
def parse_id_results(results):
        parsed_results = []
        plumb_list = []
        for result in results:
            yelp_dict = {}
            yelp_dict["biz_id"] = result['id']
            parsed_results.append(yelp_dict)
        
        for plumb in parsed_results:
            plumb_tuple = (plumb['biz_id'])
            plumb_list.append(plumb_tuple)
        return plumb_list

In [549]:
num = response.json()['total']
print('{} total matches found.'.format(num))
cur = 0
while cur < num and cur < 1000:
    url_params['offset'] = cur
    results = yelp_call(url_params, api_key)
    parsed_id_results = parse_id_results(results)
    cur = 50 + cur
    time.sleep(1) #Wait a second
    

50 total matches found.


In [571]:
cursor.execute('''SELECT biz_id FROM Plumb_biz;''')
biz_ids = cursor.fetchall()

In [572]:
biz_ids

[('-CsL219aKfHi7iPw0YSVEQ',),
 ('-GqZkEMSA2O-7wc6nEiG4w',),
 ('-YtGo9TQjaEV7K3gPFmQuQ',),
 ('00wogt3EfiEJW2T0weSr1w',),
 ('0JES2rLBuWvsfQ249PpWbg',),
 ('0m1dRGzPNlx_JLKo6lRaTQ',),
 ('1Bq3rwwe7vPEZvUA3Q9PUw',),
 ('1Eic2ZXa4BUYZfJb65WO7g',),
 ('1M5qDSvsfNogoZeXYx2icg',),
 ('1pnxGWn5aOTV2LKjC91DAw',),
 ('1Sefmcnb_FdvQrsALgFEcQ',),
 ('2n1uXZtrN2vIY9iKku6zTw',),
 ('2rMHF-tMnF_I_PCN0MM-UQ',),
 ('2U-ekZMq7IFin7JvY-RSNQ',),
 ('2_j3xkRZXsI1_WIVJsacsg',),
 ('3020UCNczgSJLYD1-JrDMw',),
 ('37gZ1A8V-WwghnSawpEi1w',),
 ('3ecwPT6aKE1sV1F4kydJsw',),
 ('3JJdXZaaI0JOZ0kUWzAEBw',),
 ('3NfHSucKngDvz24lRnDNjA',),
 ('4-BL7dutEIm8WQNvsvPqdw',),
 ('45hg5t6uHHzVNVxx4h9Cnw',),
 ('47eRRdi0QEeHlP0BGZKtNQ',),
 ('4K7AoV3ERYL7T2allzERnA',),
 ('4oiVR8uY1VtUA06PJ5bA_Q',),
 ('4QLDHUwDRlPlfjW_KTbyDQ',),
 ('4uCT_ZMzv7jUojQgt1qzkw',),
 ('4VZoPBVtKRqt7bowq5iNzA',),
 ('4Zz720ccHd7_scM_dEFH8w',),
 ('51AwoyBg4y5pqwAw1RsUyg',),
 ('5fvwivv1vcZNOYWpgURiRg',),
 ('5T6_inkams7H4hSoWqny9w',),
 ('5UxKiYOwkjp5H2RsYVTF_g',),
 ('5VFuLOW

In [503]:
def parse_results(results):
        parsed_results = []
        plumb_list = []
        for result in results:
            yelp_dict = {}
            yelp_dict["biz_id"] = result['id']
            yelp_dict["biz_name"] = result['name']
            yelp_dict["rating"] = result['rating']
            yelp_dict["review_count"] = result['review_count']
            parsed_results.append(yelp_dict)
        
        for plumb in parsed_results:
            plumb_tuple = (plumb['biz_id'], plumb['biz_name'], 
                     plumb['rating'], plumb['review_count'])
            plumb_list.append(plumb_tuple)
        return plumb_list
    
    


In [ ]:
# def parse_results_reviews(parsed_results)
        
#     for result in parsed_results:
        

In [302]:
def db_insert(data):
        insert_statement = "INSERT INTO Plumb_biz (biz_id, biz_name, rating, review_count) VALUES (%s, %s, %s, %s)"
        cursor.executemany(insert_statement, data)
        cnx.commit()

In [504]:
x = yelp_call(url_params, api_key)
parse_results(x)

[('tXtXY7r8NXPk0-MKJlrOjQ', 'Friedman Irwin & Son', 4.5, 50),
 ('1M5qDSvsfNogoZeXYx2icg', 'Pipe Monkeys Sewer and Drain', 5.0, 28),
 ('hlMkCkl-8wmFD57rD_JEJg', 'Dov Sewer and Drain', 4.5, 56),
 ('a6TKGWUFop1T-nlEIi8aNA', "Sam's Plumbing Services", 4.5, 19),
 ('g5kfI33jirjUxeYPK6DU_w', 'Michael Donahue Plumbing & Heating', 5.0, 25),
 ('TYdz9vpLiXFF7-kbwbx1cQ', 'Parkset Plumbing', 4.5, 20),
 ('4PJwxWz0V6lcLjPJ-97H0A',
  'Alexanders Proficient Heating & Cooling',
  4.0,
  38),
 ('3020UCNczgSJLYD1-JrDMw', 'Four Season Piping & Mechanical', 4.5, 24),
 ('QzpYYXgFsz7aCLgm1goDaA', 'The Original 718 Sewer & Drain', 4.5, 172),
 ('ZnMyiLNJqInJbQ1n-zpSSA', 'JPeral Piping & Heating Sewer Service', 4.5, 45),
 ('r-UxNIwsZH1KSnMkkHhZvQ', 'Franco Belli Plumbing & Heating & Sons', 4.5, 32),
 ('dNdqqq7xqTzEYSDybMyS-Q', 'JCA Mechanical', 4.5, 126),
 ('han8zgVwsxnd_fHrco8P9w', 'Vuksani Plumbing & Heating', 5.0, 5),
 ('p4f8SQpBDAUo2pl9TDnUeg', 'The Clogfather Sewer & Drain', 5.0, 3),
 ('DzgjPAeTGA8ncokuErTG

In [303]:
num = response.json()['total']
print('{} total matches found.'.format(num))
cur = 0
    
while cur < num and cur < 1000:
    url_params['offset'] = cur
    results = yelp_call(url_params, api_key)
    parsed_results = parse_results(results)
    db_insert(parsed_results)
    time.sleep(1) #Wait a second
    cur = 50 + cur
    

437 total matches found.


In [ ]:
results1 = yelp_call(url_params, api_key)
tuple_biz_results = parse_results(results1)
#get a list of the biz ids 
# def parse_results_reviews():
#       yelp_reviews = []
#           for result in prev_results_from_before:
#             yelp_dict = {}
#             id = result["biz_id"]
#             data = yelp_call_reviews(api_key,Url_thing,id)
#             yelp_dict["id"] = data["id"]
#             yelp_dict["reviews"] = data["reviews"]
#             yelp_reviews.append(yelp_dict)
#             time.sleep(.5sec)

## Part 2: Create ETL pipeline for the business data from the API

In [305]:
locale = 'en_US'
url_params = {  'term': locale.replace(' ', '+')
             }

In [555]:
url = 'https://api.yelp.com/v3/businesses/{id}/reviews'
client_id = "kSJKHzaATCEw4bxY9tJGwA"
api_key = "xZce1e9e3JthFVYS0eQm1Aa6thcOEaQ2N2hzi3PJsJrv6ga4hinQMYsPWTBYQYRikD7sN9H7HQJV9sDPBoIf-m2Ko-0rw5ouSZhGgUtQDYA5_5hEipYddHf7Ct2xXXYx"

In [ ]:
url = https://api.yelp.com/v3/businesses/{id}/reviews
headers = {'Authorization': 'Bearer {}'.format(api_key)}
response = requests.get(url, headers=headers, params=url_params)
   

In [ ]:
def yelp_call_reviews(url_params, api_key):
    url = ('https://api.yelp.com/v3/businesses/{}/reviews'.format(biz_id))
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['reviews']
    return data

In [ ]:

    
#get a list of the biz ids 
# def parse_results_reviews(prev_results_from_before):
#   yelp_reviews = []
#   for result in prev_results_from_before:
#     yelp_dict = {}
#     id = result["biz_id"]
#     data = yelp_call_reviews(api_key,Url_thing,id)
#     yelp_dict["id"] = data["id"]
#     yelp_dict["reviews"] = data["reviews"]
#     yelp_reviews.append(yelp_dict)
#     time.sleep(.5sec)
        
    "  rev_id int(71) NOT NULL,"
    "  rev_rating int(8) NOT NULL,"
    "  name varchar(40) NOT NULL,"
    "  user_id varchar(40) NOT NULL,"
    "  review_text varchar(5000) NOT NULL,"
    "  rev_date date NOT NULL,"
    "  biz_id int(40) NOT NULL,"
    "  PRIMARY KEY (rev_id)"

In [ ]:
def parse_results_for_id(results):
        parsed_results = []
        plumb_list = []
        for result in results:
            yelp_dict = {}
            yelp_dict["biz_id"] = result['id']
            yelp_dict["biz_name"] = result['name']
            yelp_dict["rating"] = result['rating']
            yelp_dict["review_count"] = result['review_count']
            parsed_results.append(yelp_dict)
        
        for plumb in parsed_results:
            plumb_tuple = (plumb['biz_id'], plumb['biz_name'], 
                     plumb['rating'], plumb['review_count'])
            plumb_list.append(plumb_tuple)
        return plumb_list
    


In [ ]:
num = response.json()['total']
print('{} total matches found.'.format(num))
cur = 0
    
while cur < num and cur < 1000:
    biz_id = [] 
    results = yelp_call(url_params, api_key)
    parsed_results = parse_results(results)
    time.sleep(1) #Wait a second
    cur = 50 + cur
    

In [505]:
def parse_id_results(results):
        parsed_results = []
        plumb_list = []
        for result in results:
            yelp_dict = {}
            yelp_dict["biz_id"] = result['id']
            parsed_results.append(yelp_dict)
        
        for plumb in parsed_results:
            plumb_tuple = (plumb['biz_id'])
            plumb_list.append(plumb_tuple)
        return plumb_list

In [507]:
list_of_ids = parse_id_results(results1)

In [510]:
url = ('https://api.yelp.com/v3/businesses/%s/reviews'% list_of_ids[0])
headers = {'Authorization': 'Bearer {}'.format(api_key)}
response = requests.get(url, headers=headers, params=url_params)
response

<Response [200]>

In [515]:
response.json()["reviews"]


[{'id': 'Yd5Cxu9YIwHVQObrLmW1tA',
  'url': 'https://www.yelp.com/biz/friedman-irwin-and-son-brooklyn?adjust_creative=kSJKHzaATCEw4bxY9tJGwA&hrid=Yd5Cxu9YIwHVQObrLmW1tA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=kSJKHzaATCEw4bxY9tJGwA',
  'text': 'Brandon did a superb job! Called him in to fix a shower and had a problem with my bathroom faucet during the time we were waiting on the shower part. Not...',
  'rating': 5,
  'time_created': '2019-06-21 13:59:41',
  'user': {'id': 'V8ftzOvcy9eO8lePGqBJBQ',
   'profile_url': 'https://www.yelp.com/user_details?userid=V8ftzOvcy9eO8lePGqBJBQ',
   'image_url': None,
   'name': 'Jeff B.'}},
 {'id': 'HYqczo-J1uG5WA3TabF54w',
  'url': 'https://www.yelp.com/biz/friedman-irwin-and-son-brooklyn?adjust_creative=kSJKHzaATCEw4bxY9tJGwA&hrid=HYqczo-J1uG5WA3TabF54w&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=kSJKHzaATCEw4bxY9tJGwA',
  'text': 'Super accommodating, onsite in a few hours and SUPERB JOB!\n\

In [346]:


headers = {'Authorization': 'Bearer {}'.format(api_key)}
response = requests.get(url, headers=headers, params=url_params)
url = 'https://api.yelp.com/v3/businesses/search'
client_id = "kSJKHzaATCEw4bxY9tJGwA"
api_key = "xZce1e9e3JthFVYS0eQm1Aa6thcOEaQ2N2hzi3PJsJrv6ga4hinQMYsPWTBYQYRikD7sN9H7HQJV9sDPBoIf-m2Ko-0rw5ouSZhGgUtQDYA5_5hEipYddHf7Ct2xXXYx"

term = 'Plumbers'
location = 'Bushwick NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50,
                'offset' : 0
             }

import json
import requests
import time

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()['businesses']
    return data

# def parse_id_results(results):
#         parsed_results = []
#         plumb_list = []
#         for result in results:
#             yelp_dict = {}
#             yelp_dict["biz_id"] = result['id']
#             parsed_results.append(yelp_dict)
        
#         for plumb in parsed_results:
#             plumb_tuple = (plumb['biz_id'])
#             plumb_list.append(plumb_tuple)
#         return plumb_list
    

def yelp_call_review(api_key, list_of_ids):
    reviews = []
    for i in list_of_ids:
        url = ('https://api.yelp.com/v3/businesses/%s/reviews'% i)
        headers = {'Authorization': 'Bearer {}'.format(api_key)}
        response = requests.get(url, headers=headers, params=url_params)
        data = response.json()['reviews']
        data["biz_id"] = i
        reviews.extend(data)
    return data


def parse_results_reviews(previous_results):
    yelp_reviews = []
    for result in previous_results:
        yelp_dict = {}
        the_id = result['biz_id']
        data = yelp_call_review(api_key,the_id)
        yelp_reviews.append(data)
        time.sleep(5)


def parse_review_results(results):
        parsed_results = []
        rev_list = []
        for result in results:
            yelp_dict = {}
            yelp_dict["rev_id"] = result['id']
            yelp_dict["rev_rating"] = result['rating']
            yelp_dict["name"] = result['user']
            yelp_dict["review_text"] = result['text']
            yelp_dict["rev_date"] = result['time_created']
            parsed_results.append(yelp_dict)
        
        for rev in parsed_results:
            rev_tuple = (rev['rev_id'], rev['rev_rating'], rev['name'], rev['review_text'], rev['rev_date'])
            rev_list.append(rev_tuple)
        return rev_list

def create_tuple(data): 
    for rev in yelp_reviews:
        rev_tuple = (rev['rev_id'], rev['rev_rating'], rev['name'], rev['review_text'], rev['rev_date'])
            rev_list.append(rev_tuple)
    return rev_list
    
def db_insert_reviews(data):
        insert_statement = "INSERT INTO Plumb_Reviews (rev_id, rev_rating, name, review_text, rev_date) VALUES (%s, %s, %s, %s, %s)"
        cursor.executemany(insert_statement, data)
        cnx.commit()


cur = 0  
while cur < 438:
    url_params['offset'] = cur
    results = yelp_call(url_params, api_key)
    biz_id = parse_results_reviews(results)
    review_results = yelp_call_review(api_key, the_id)
    parsed_review_results = parse_review_results(review_results)
    db_insert_reviews(parsed_review_results)
    time.sleep(1) #Wait a second
    cur = 50 + cur
    
    

TypeError: list indices must be integers or slices, not str

In [442]:
biz_id = ['tXtXY7r8NXPk0-MKJlrOjQ',
 '1M5qDSvsfNogoZeXYx2icg',
 'hlMkCkl-8wmFD57rD_JEJg']

api_key = "xZce1e9e3JthFVYS0eQm1Aa6thcOEaQ2N2hzi3PJsJrv6ga4hinQMYsPWTBYQYRikD7sN9H7HQJV9sDPBoIf-m2Ko-0rw5ouSZhGgUtQDYA5_5hEipYddHf7Ct2xXXYx"

length = len(biz_id)
for i in range(0,length):
    print(i)

0
1
2


In [470]:
def yelp_call_review(api_key):
    length = len(biz_id)   
    for i in range(0,length):
        url = ('https://api.yelp.com/v3/businesses/%s/reviews'% biz_id[i])
        headers = {'Authorization': 'Bearer {}'.format(api_key)}
        response = requests.get(url, headers=headers, params=url_params)
        data = response.json()['reviews']
    return data



In [471]:
x = yelp_call_review(api_key)

def parse_review_results(results):
        parsed_results = []
        rev_list = []
        for result1 in results:
            for result in result1:
                yelp_dict = {}
                yelp_dict["rev_id"] = result['id']
                yelp_dict["rev_rating"] = result['rating']
                yelp_dict["name"] = result['user']
                yelp_dict["review_text"] = result['text']
                yelp_dict["rev_date"] = result['time_created']
                parsed_results.append(yelp_dict)
        
            for rev in parsed_results:
                rev_tuple = (rev['rev_id'], rev['rev_rating'], rev['name'], rev['review_text'], rev['rev_date'])
                rev_list.append(rev_tuple)
            return rev_list

previous_results = parse_review_results(results)
        
def parse_results_reviews(previous_results):
    yelp_reviews = []
    for result in previous_results:
        yelp_dict = {}
        the_id = result['biz_id']
        data = yelp_call_reviews(api_key,the_id)
        yelp_reviews.append(data)
        time.sleep(.5sec)
        
#         yelp_dict['id'] = data['id']
#         yelp_dict["reviews"] = data["reviews"]
#         yelp_reviews.append(yelp_dict)
#         time.sleep(.5sec)

In [486]:
def parse_results_for_id(results):
        parsed_results = []
        for result in results:
            yelp_dict = {}
            yelp_dict["biz_id"] = result['id']
            yelp_dict["biz_name"] = result['name']
            yelp_dict["rating"] = result['rating']
            yelp_dict["review_count"] = result['review_count']
            parsed_results.append(yelp_dict)
        return parsed_results
    


In [495]:
previous_results = parse_results_for_id(results1)
previous_results

[{'biz_id': 'tXtXY7r8NXPk0-MKJlrOjQ',
  'biz_name': 'Friedman Irwin & Son',
  'rating': 4.5,
  'review_count': 50},
 {'biz_id': '1M5qDSvsfNogoZeXYx2icg',
  'biz_name': 'Pipe Monkeys Sewer and Drain',
  'rating': 5.0,
  'review_count': 27},
 {'biz_id': 'hlMkCkl-8wmFD57rD_JEJg',
  'biz_name': 'Dov Sewer and Drain',
  'rating': 4.5,
  'review_count': 57},
 {'biz_id': 'a6TKGWUFop1T-nlEIi8aNA',
  'biz_name': "Sam's Plumbing Services",
  'rating': 4.5,
  'review_count': 19},
 {'biz_id': 'TYdz9vpLiXFF7-kbwbx1cQ',
  'biz_name': 'Parkset Plumbing',
  'rating': 4.5,
  'review_count': 20},
 {'biz_id': 'g5kfI33jirjUxeYPK6DU_w',
  'biz_name': 'Michael Donahue Plumbing & Heating',
  'rating': 5.0,
  'review_count': 25},
 {'biz_id': '3020UCNczgSJLYD1-JrDMw',
  'biz_name': 'Four Season Piping & Mechanical',
  'rating': 4.5,
  'review_count': 24},
 {'biz_id': 'QzpYYXgFsz7aCLgm1goDaA',
  'biz_name': 'The Original 718 Sewer & Drain',
  'rating': 4.5,
  'review_count': 172},
 {'biz_id': 'PvuChiGcd-T-KIPRl

In [491]:
def yelp_call_review(api_key, the_id):
        url = ('https://api.yelp.com/v3/businesses/%s/reviews'% the_id)
        headers = {'Authorization': 'Bearer {}'.format(api_key)}
        response = requests.get(url, headers=headers, params=url_params)
        data = response.json()['reviews']
        return data


In [496]:
def parse_results_reviews(previous_results):
    yelp_reviews = []
    for result in previous_results:
        yelp_dict = {}
        the_id = result['biz_id']
        data = yelp_call_review(api_key,the_id)
        yelp_reviews.append(data)
        time.sleep(5)
        
def create_tuple(data): 
    for rev in yelp_reviews:
        rev_tuple = (rev['rev_id'], rev['rev_rating'], rev['name'], rev['review_text'], rev['rev_date'])
            rev_list.append(rev_tuple)
    return rev_list
    

SyntaxError: invalid syntax (<ipython-input-496-4bc5846315ce>, line 10)

In [493]:
parse_results_reviews(previous_results)

## Part 3: Create ETL pipeline for the restaurant review data from the API

In [ ]:
# write a SQL query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

In [654]:
def yelp_call_review(api_key, biz_ids):
    for i in biz_ids:
        url = ('https://api.yelp.com/v3/businesses/%s/reviews'% i)
        headers = {'Authorization': 'Bearer {}'.format(api_key)}
        response = requests.get(url, headers=headers, params=url_params)
        data = response.json()['reviews']
        time.sleep(.5)
        continue



In [655]:
new_biz_ids = biz_ids[0:3]

In [656]:
reviews = yelp_call_review(api_key, new_biz_ids)


In [ ]:
soup.select

In [653]:
reviews

[{'id': '53kveCv7IK4AMbLkFnWyQw',
  'url': 'https://www.yelp.com/biz/cilento-pipeline-plumbing-and-heating-oceanside?adjust_creative=kSJKHzaATCEw4bxY9tJGwA&hrid=53kveCv7IK4AMbLkFnWyQw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=kSJKHzaATCEw4bxY9tJGwA',
  'text': 'After Hurricane Sandy I put up a Baxi Luna tankless Boiler to replace the traditional one I lost in the storm. Well after 2 years the machine started to...',
  'rating': 5,
  'time_created': '2015-02-08 05:03:36',
  'user': {'id': 'HwVIAJQbJiStL4Wyl4ocMw',
   'profile_url': 'https://www.yelp.com/user_details?userid=HwVIAJQbJiStL4Wyl4ocMw',
   'image_url': 'https://s3-media4.fl.yelpcdn.com/photo/Z89EmKHBx01g5R4JiNUebw/o.jpg',
   'name': 'Roz S.'}},
 {'id': '4icZX_J2NbA3B26Ik57qNg',
  'url': 'https://www.yelp.com/biz/cilento-pipeline-plumbing-and-heating-oceanside?adjust_creative=kSJKHzaATCEw4bxY9tJGwA&hrid=4icZX_J2NbA3B26Ik57qNg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=kS

In [610]:
reviews = yelp_call_review(api_key, biz_ids)

KeyError: 'reviews'

In [606]:
def parse_review_results(results):
        parsed_results = []
        rev_list = []
        for result in results['reviews']:
            yelp_dict = {}
            yelp_dict["rev_id"] = result['id']
            yelp_dict["rev_rating"] = result['rating']
            yelp_dict["name"] = result['user']
            yelp_dict["review_text"] = result['text']
            yelp_dict["rev_date"] = result['time_created']
            parsed_results.append(yelp_dict)
        
        for rev in parsed_results:
            rev_tuple = (rev['rev_id'], rev['rev_rating'], rev['name'], rev['review_text'], rev['rev_date'])
            rev_list.append(rev_tuple)
        return rev_list

In [607]:
parse_review_results(reviews)

[('5uZVvDW8MxUH9I4sevqNSg',
  5,
  {'id': 'kyd4YipQY99I7uwxujFivA',
   'profile_url': 'https://www.yelp.com/user_details?userid=kyd4YipQY99I7uwxujFivA',
   'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/TCuThgpRAmZFOrtwk5rTHg/o.jpg',
   'name': 'Dawn H.'},
  'These guys are absolutely amazing!!! I highly recommend them. I had a pipe break on on my boiler, on a Saturday night.  They were very professional on the...',
  '2018-03-16 17:40:09'),
 ('zkkaCiG1q3Nkh7Wo9EeQLg',
  5,
  {'id': '-WEGyk8InhSENhgW-2BzkA',
   'profile_url': 'https://www.yelp.com/user_details?userid=-WEGyk8InhSENhgW-2BzkA',
   'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/PMbtMMNooplqqmHNLR-BPQ/o.jpg',
   'name': 'Matt D.'},
  'Prior to calling these guys I went through three different plumbers!!! Each one attempted to clear a clog that was causing my basement to flood.  Each one...',
  '2018-02-20 17:02:01'),
 ('t5ICb-3mR7lqUGu52iEMyg',
  1,
  {'id': 'rozR76USGkaCLDysQ5acTw',
   'profile_url': 'https://w

In [ ]:
def db_insert_reviews(data):
        insert_statement = "INSERT INTO Plumb_Reviews (rev_id, rev_rating, name, review_text, rev_date) VALUES (%s, %s, %s, %s, %s)"
        cursor.executemany(insert_statement, data)
        cnx.commit()


In [ ]:
# write a function to insert the parsed data into the reviews table

In [ ]:
# combine the functions above into a single script  

## Part 4: Write SQL queries that will answer the questions posed. 

In [ ]:
# create connection

In [ ]:
# execute SQL queries

# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

In [ ]:
def parse_results(results):
    
    return parsed_results

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time



def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results = yelp_call(url_params, api_key)
        parsed_results = parse_results(results)
        db_insert(parsed)
        time.sleep(1) #Wait a second
        cur += 50

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id INT AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```

### Using DB:
    
For this lab, you can either store the data on one DB or put in on both of the partners DBs. If you decide to put it on one DB, you want to make sure both partners have access to it.  To do this you want to add a user to your DB.  

[how to add a new user](https://howchoo.com/g/mtm3zdq2nzv/how-to-add-a-mysql-user-and-grant-privileges)